# Recitation 1: a crash course in Julia and JuMP

<a href="http://julialang.org"><img src="figures/julia.png" alt="Julia" style="width: 150px;"/></a>
<a href="http://jump.dev"><img src="figures/JuMP-logo.png" alt="JuMP" style="width: 150px;"/></a>

## 1. Why Julia/JuMP?

- Julia is a "high-level, high-performance dynamic programming language for technical computing." Think the linear algebra power of Matlab, with the speed of C and the readability of Python.
- JuMP is a library that allows us to easily formulate optimization problems and solve them using a variety of solvers. It provides an easy interface to implement advanced optimization techniques.
- Check out this [talk](https://github.com/johnfgibson/whyjulia/blob/master/1-whyjulia.ipynb]) for more details on why Julia is awesome.

## 2. Julia basics

### 2.1 Jupyter

#### What is a Jupyter notebook?

- Jupyter notebooks are **documents** (like a Word document) that can contain and run code.
- They were originally created for Python as part of the IPython project, and adapted for Julia by the **IJulia** project.
- They are very useful to **prototype**, draw **plots**, or even for teaching material like this one.
- The document relies only on a modern browser for rendering, and can easily be **shared**.

#### How do I even open this file?

Once Julia is installed, start julia and just run the following commands to install the `IJulia` package.
```jl
using Pkg
Pkg.install("IJulia")
```
This should work on its own. If there is any issue, check out the [IJulia website](https://github.com/JuliaLang/IJulia.jl).

Once IJulia is installed, go to the directory containing the notebook file (`Recitation 1.ipynb`), start julia and run:
```jl
using IJulia
notebook()
```
A webpage should open automatically, just click on the notebook to load it.

#### Navigating the notebook

- Click `Help -> User Interface Tour` for a guided tour of the notebook interface.
- Each notebook is composed of **cells**, that either contain code or text (`Markdown`).
- You can edit the content of a cell by double-clicking on it (_Edit Mode_).

When you are not editing a cell, you are in _Command mode_ and can edit the structure of the notebook (cells, name, options...)

- Create a cell by:
    - Clicking `Insert -> Insert Cell`
    - Pressing `a` or `b` in Command Mode
    - Pressing `Alt+Enter` in Edit Mode
- Delete a cell by:
    - Clicking `Edit -> Delete Cell`
    - Pressing `dd`
- Execute a cell by:
    - Clicking `Cell -> Run`
    - Pressing `Ctrl+Enter`
    - Pressing `Shift+Enter` (this will also move your focus to the next cell)
    
Other functions:
- Undo last text edit with `Ctrl+z` in Edit Mode
- Undo last cell manipulation with `z` in Command Mode
- Save notebook with `Ctrl+s` in Edit Mode
- Save notebook with `s` in Command Mode

Though notebooks rely on your browser to work, they do not require an internet connection (except for math rendering).

### 2.2 How to Julia

Julia, as a dynamic language, can simply be used as a calculator:

In [1]:
1+1

2

In [2]:
sin(exp(2*pi)+sqrt(3))

-0.01136232398070678

The key building blocks of Julia code are variables:

In [3]:
a = 1
b = 2
# This is a comment 
c = a^2 + b^3 

9

Julia supports the usual `if`, `while` and `for` structures:

In [4]:
if c >= 10
    print("Yes")
else
    print("No")
end

No

In [5]:
i = 1
while i <= 5
    println("Why, hello!") # Print with a new line
    i += 1
end

Why, hello!
Why, hello!
Why, hello!
Why, hello!
Why, hello!


In [6]:
for i = 1:3
    print("$i banana") # '$' can be used to insert variables into text
    if i>1
        print("s")
    end
    println() # Just a new line
end

1 banana
2 bananas
3 bananas


**Do not worry about writing loops**: in Julia, they are as fast as writing vectorized code, and sometimes faster!

**Arrays** (list of numbers) are at the core of research computing and Julia's arrays are extremely optimized.

In [7]:
myList = [6, 7, 8]

3-element Array{Int64,1}:
 6
 7
 8

Array indexing starts with 1 in Julia, and arrays are mutable.

In [8]:
@show myList[1]
myList[3] = 4
@show myList;

myList[1] = 6
myList = [6, 7, 4]


A 2-dimensional array is a Matrix

In [9]:
A = [1 2 3
     2 1 2
     3 2 1]

A = [1 2 3; 2 1 2; 3 2 1] #same thing

3×3 Array{Int64,2}:
 1  2  3
 2  1  2
 3  2  1

## 2.3 Reading data - CSV and DataFrames

You can install these packages with:

In [10]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
  Installed Reexport ─────────── v1.0.0
  Installed SortingAlgorithms ── v0.3.1
  Installed Formatting ───────── v0.4.2
  Installed Compat ───────────── v3.25.0
  Installed StructTypes ──────── v1.4.0
  Installed OrderedCollections ─ v1.4.0
  Installed Crayons ──────────── v4.0.4
  Installed InvertedIndices ──── v1.0.0
  Installed Missings ─────────── v0.4.5
  Installed DataStructures ───── v0.18.9
  Installed DataFrames ───────── v0.22.5
  Installed CategoricalArrays ── v0.9.3
  Installed PrettyTables ─────── v0.11.1
Updating `~/.julia/environments/v1.5/Project.toml`
  [a93c6f00] + DataFrames v0.22.5
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [324d7699] + CategoricalA

In [11]:
using DataFrames, CSV

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1278


We're going to load the data for our optimization example, the transportation problem, where factories and markets are both located in the 2D plane.
- `data/supply.csv` has one row per factory, with columns for the (x, y) coordinates, and a column for the capacity
- `data/demand.csv` has one row per market, with columns for the (x, y) coordinates, and a column for the demand

In [12]:
supply = CSV.read("data/supply.csv", DataFrame)
demand = CSV.read("data/demand.csv", DataFrame);
first(demand, 5)

,x,y,demand
,Float64,Float64,Int64
1,4.10387,7.01562,5
2,8.62726,2.90866,1
3,8.04627,4.87151,3
4,4.48929,3.90493,4
5,1.73366,3.30187,4


In [13]:
Pkg.add("JuMP")

  Resolving package versions...
  Installed IniFile ────────────────────── v0.5.0
  Installed CodecZlib ──────────────────── v0.7.0
  Installed Calculus ───────────────────── v0.5.1
  Installed DiffResults ────────────────── v1.0.3
  Installed MacroTools ─────────────────── v0.5.6
  Installed Bzip2_jll ──────────────────── v1.0.6+5
  Installed Zlib_jll ───────────────────── v1.2.11+18
  Installed MutableArithmetics ─────────── v0.2.14
  Installed URIs ───────────────────────── v1.2.0
  Installed StaticArrays ───────────────── v1.0.1
  Installed ChainRulesCore ─────────────── v0.9.29
  Installed JuMP ───────────────────────── v0.21.6
  Installed MathOptInterface ───────────── v0.9.20
  Installed OpenSpecFun_jll ────────────── v0.5.3+4
  Installed DiffRules ──────────────────── v1.0.2
  Installed CodecBzip2 ─────────────────── v0.7.2
  Installed JSONSchema ─────────────────── v0.3.3
  Installed ZipFile ────────────────────── v0.9.3
  Installed NetworkOptions ─────────────── v1.2.0
  Inst

In [14]:
Pkg.add("Gurobi")

  Resolving package versions...
  Installed CEnum ── v0.4.1
  Installed Gurobi ─ v0.9.9
Updating `~/.julia/environments/v1.5/Project.toml`
  [2e9cd046] + Gurobi v0.9.9
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [fa961155] + CEnum v0.4.1
  [2e9cd046] + Gurobi v0.9.9
   Building Gurobi → `~/.julia/packages/Gurobi/qk7lG/deps/build.log`
┌ Error: Error building `Gurobi`: 
│ ERROR: LoadError: Unable to locate Gurobi installation. If the advice above did not help,
│ open an issue at https://github.com/jump-dev/Gurobi.jl and post the full
│ print-out of this diagnostic attempt.
│ 
│ Stacktrace:
│  [1] error(::String) at ./error.jl:33
│  [2] top-level scope at /Users/leonardboussioux/.julia/packages/Gurobi/qk7lG/deps/build.jl:161
│  [3] include(::String) at ./client.jl:457
│  [4] top-level scope at none:5
│ in expression starting at /Users/leonardboussioux/.julia/packages/Gurobi/qk7lG/deps/build.jl:155
│ 
│ **Unable to locate Gurobi installation. Running some common diagnostics.**
│ 

## 3. Basics of JuMP

Now we will use this data to formulate and solve the transportation problem. First, we need to install a solver. A good choice is the Gurobi solver. You can follow [these instructions](https://github.com/jump-dev/Gurobi.jl) to install both Gurobi and its Julia wrapper `Gurobi.jl`.

Then we can load JuMP and Gurobi.

In [5]:
using JuMP, Gurobi

┌ Info: Recompiling stale cache file /Users/leobix/.julia/compiled/v1.2/Gurobi/do9v6.ji for Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1240
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



We're going to use JuMP to "translate" our transportation problem (see slides) into something that Gurobi can solve.

In [6]:
"Function to build the transportation model, returns model and decision variable handles"
function build_transportation_model(supply::DataFrame, demand::DataFrame)
    # initialize the model, and specify the solver
    model = Model(Gurobi.Optimizer)
    # Decision variables
    @variable(model, x[i = 1:nrow(supply), j = 1:nrow(demand)] >= 0)
    # Capacity constraint
    @constraint(model, capacity_constraint[i=1:nrow(supply)],
                sum(x[i, j] for j = 1:nrow(demand)) <= supply[i, "capacity"])
    # Demand constraint
    @constraint(model, demand_constraint[j=1:nrow(demand)],
                sum(x[i, j] for i = 1:nrow(supply)) >= demand[j, "demand"])
    # Objective
    @objective(model, Min,
               sum(x[i, j] *
                   sqrt((supply[i, "x"] - demand[j, "x"]) ^ 2 + (supply[i, "y"] - demand[j, "y"]) ^ 2)
                   for i = 1:nrow(supply), j=1:nrow(demand)))
    return model, x
end

build_transportation_model

We can now build the optimization model. Notice that Jupyter can display the model (but beware output overload for large models).

In [ ]:
model, x = build_transportation_model(supply, demand)

In [ ]:
model

In [ ]:
x

Now we can solve the model using the `optimize!` command. The `!` is a Julia convention that indicates that the function modifies its argument (in this case, by solving the optimization problem).

In [ ]:
optimize!(model)

Now we can extract the optimal objective:

In [ ]:
objective_value(model)

We can also obtain the optimal variable values:

In [ ]:
value(x[1, 4])

In [ ]:
[value(x[i, j]) for i=1:nrow(supply), j=1:nrow(demand)]